In [63]:
from ctr.importxml import *
import lxml
from bs4 import BeautifulSoup
from pathlib import Path

In [64]:
p = 'F:\\Reporting\\CTR\\2021\\September'
path = Path(p)
file_list = get_file_list(path)
xml_list = get_xml_list(file_list)
merged_xml = merge_xml(xml_list)
transaction_list = merged_xml.findall('transaction')


def get_account_details(ac_node):
    branch = ac_node.find('branch').text
    ac_no = ac_node.find('account').text
    ac_name = ac_node.find('account_name').text
    ac_type = ac_node.find('personal_account_type').text
    return branch, ac_no, ac_name, ac_type

def get_person_details(person_node):
    name = person_node.find('first_name').text+" "+person_node.find('last_name').text
    return name

location_list = []
description_list = []
date_list = []
mode_list = []
amount_list = []
from_funds_code_list = []
from_branch_list = []
from_ac_number_list = []
from_ac_name_list = []
from_ac_type_list = []
from_person_name_list = []
to_funds_code_list = []
to_branch_list = []
to_ac_number_list = []
to_ac_name_list = []
to_ac_type_list = []
to_person_name_list = []

#transaction_list

In [65]:
def get_df_from_xml(transaction_list):
    for tr in transaction_list:
        location_list.append(tr.find("transaction_location").text)
        description_list.append(tr.find("transaction_description").text)
        date_list.append(tr.find("date_transaction").text)
        mode_list.append(tr.find("transmode_code").text)
        amount_list.append(tr.find("amount_local").text)
        ############### from part #####################
        if tr.find("t_from_my_client"):
            frm = tr.find("t_from_my_client")
            from_funds_code = frm.find("from_funds_code").text
            from_funds_code_list.append(from_funds_code)
            # from_country = frm.find('from_country')
            if frm.find("from_account"):
                ac_node = frm.find("from_account")
                from_branch, from_ac, from_ac_name, from_ac_type = get_account_details(ac_node)
                from_branch_list.append(from_branch)
                from_ac_number_list.append(from_ac)
                from_ac_name_list.append(from_ac_name)
                from_ac_type_list.append(from_ac_type)
                
                ##### padding for person #########
                from_person_name_list.append('')
                
            elif frm.find("from_person"):
                person_node = frm.find("from_person")
                from_person_name = get_person_details(person_node)
                from_person_name_list.append(from_person_name)
                
                ################# padding for account ##################
                from_branch_list.append('')
                from_ac_number_list.append('')
                from_ac_name_list.append('')
                from_ac_type_list.append('')
            else:
                pass
        else:
            frm = tr.find("t_from")
            from_funds_code = frm.find("from_funds_code").text
            from_funds_code_list.append(from_funds_code)
            # from_country = frm.find('from_country')
            if frm.find("from_account"):
                ac_node = frm.find("from_account")
                from_branch, from_ac, from_ac_name, from_ac_type = get_account_details(ac_node)
                from_branch_list.append(from_branch)
                from_ac_number_list.append(from_ac)
                from_ac_name_list.append(from_ac_name)
                from_ac_type_list.append(from_ac_type)
                
                ##### padding for person #########
                from_person_name_list.append('')
                
            elif frm.find("from_person"):
                person_node = frm.find("from_person")
                from_person_name = get_person_details(person_node)
                from_person_name_list.append(from_person_name)
                
                ################# padding for account ##################
                from_branch_list.append('')
                from_ac_number_list.append('')
                from_ac_name_list.append('')
                from_ac_type_list.append('')
            else:
                pass
#################### to part#######################

        if tr.find("t_to_my_client"):
            to = tr.find("t_to_my_client")
            to_funds_code = to.find("to_funds_code").text
            to_country = to.find("to_country").text
            if to.find("to_account"):
                ac_node = to.find("to_account")
                to_branch, to_ac, to_ac_name, to_ac_type = get_account_details(ac_node)
                to_branch_list.append(to_branch)
                to_ac_number_list.append(to_ac)
                to_ac_name_list.append(to_ac_name)
                to_ac_type_list.append(to_ac_type)
                
                ##### padding for person #########
                to_person_name_list.append(to_person_name)
                
            elif to.find("to_person"):
                person_node = to.find("to_person")
                to_person_name = get_person_details(person_node)
                to_person_name_list.append(to_person_name)
                
                ############### Padding for account ###############
                to_branch_list.append('')
                to_ac_number_list.append('')
                to_ac_name_list.append('')
                to_ac_type_list.append('')
            else:
                pass
        else:
            to = tr.find("t_to")
            to_funds_code = to.find("to_funds_code").text
            to_country = frm.find("to_country").text
            if to.find("to_account"):
                ac_node = to.find("to_account")
                to_branch, to_ac, to_ac_name, to_ac_type = get_account_details(ac_node)
                to_branch_list.append(to_branch)
                to_ac_number_list.append(to_ac)
                to_ac_name_list.append(to_ac_name)
                to_ac_type_list.append(to_ac_type)
                
                ##### padding for person #########
                to_person_name_list.append('')
                
            elif to.find("to_person"):
                person_node = to.find("to_person")
                to_person_name = get_person_details(person_node)
                to_person_name_list.append(to_person_name)
                
                ############### Padding for account ###############
                to_branch_list.append('')
                to_ac_number_list.append('')
                to_ac_name_list.append('')
                to_ac_type_list.append('')
            else:
                pass



In [66]:
def fill_with_empty_str(length, ls):
    if ls:
        return ls
    else:
        ls = ['']*length
        return ls

get_df_from_xml(transaction_list)

length = len(location_list)


dictionary = {#'Branch Code':code_list,
            'location': location_list,
            'description': description_list,
            'date': date_list,
            'mode': mode_list,
            'amount':amount_list,
            'from_fund_code': from_funds_code_list,
            'from_branch': from_branch_list,
            'from_ac_number': from_ac_number_list,
            'from_ac_name': from_ac_name_list,
            'from_ac_type': from_ac_type_list,
            'from_person': from_person_name_list,
            'to_fund_code':to_funds_code_list,
            'to_branch': to_branch_list,
            'to_ac_number':to_ac_number_list,
            'to_ac_name': to_ac_name_list,
            'to_ac_type': to_ac_type_list,
            'to_person': to_person_name_list,
            }
df = pd.DataFrame(dictionary)
#df['from_ac_prefix'] = df['from_ac_number'].str[:2]
#df['to_ac_prefix'] = df['to_ac_number'].str[:2]
#location_list

ValueError: All arrays must be of the same length

In [ ]:
df[(df['from_branch']=='O R NIZAM ROAD BRANCH-0002,') | (df['to_branch']=='O R NIZAM ROAD BRANCH-0002,')].to_clipboard()


INSTALLED VERSIONS
------------------
commit           : 945c9ed766a61c7d2c0a7cbb251b6edebf9cb7d5
python           : 3.7.9.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
Version          : 10.0.19041
machine          : AMD64
processor        : Intel64 Family 6 Model 165 Stepping 5, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : None.None

pandas           : 1.3.4
numpy            : 1.21.3
pytz             : 2018.5
dateutil         : 2.7.3
pip              : 21.2.4
setuptools       : 57.0.0
Cython           : None
pytest           : None
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : 1.0.5
lxml.etree       : 4.4.2
html5lib         : 1.0.1
pymysql          : None
psycopg2         : 2.7.6.1 (dt dec pq3 ext lo64)
jinja2           : 3.0.1
IPython          : 7.23.1
pandas_datareader: None
bs4              : 4.8.2
bottleneck       : No

In [6]:
tr = merged_xml.find('transactionnumber')

In [11]:
df.to_clipboard()